## Cleaning Data

In this notebook we start the analizing the two dataset and correct errors


In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pathlib import Path
from scipy.stats.mstats import winsorize


## Taxi Dataset

We start from the taxi dataset

In [ ]:
df = pd.read_csv(Path("data/Chicago_Taxi_Trips_2020_10percent.csv")) # this open the csv dataset considering the path
#print(df.head()) #first 5 lines

df.columns = [c.replace(' ', '_') for c in df.columns]  #replaces spaces in columns with underscores

In [ ]:
df.head()

In [ ]:
df.shape

### Data preparation - Time

In [ ]:
#This code recognises AM/PM difference
df.Trip_Start_Timestamp = pd.to_datetime(df.Trip_Start_Timestamp, format='%m/%d/%Y %I:%M:%S %p')
df.Trip_End_Timestamp = pd.to_datetime(df.Trip_End_Timestamp, format='%m/%d/%Y %I:%M:%S %p')
#This line excludes trips that End before Starting (90 trips)
df = df[df.Trip_End_Timestamp >= df.Trip_Start_Timestamp]

In [ ]:
#TODO ci sarebbe da controllare se la durata in secondi sia la stessa della differenza fra partenza e arrivo
#df=df[(df.Trip_End_Timestamp-df.Trip_Start_Timestamp)==pd.to_timedelta(df.Trip_Seconds * 10 ** 9)]

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.head()

### Data preparation - MV



We count the number of missing values for each columns

In [ ]:
df.isnull().sum()

Using the description given from the City of Chicago we can assume that there are
from 23930 to 30916 trips from (or to) locations outside Chicago.

We firstly check if there are totally empty rows. We will remove them.

In [ ]:
df.dropna(how='all').shape

Turns out that the all rows have at least one valid value

### Statistics

In [ ]:
from scipy.stats import median_abs_deviation #as median_abs_deviation

The following function will help us to compute some statics useful to analyze our data.
???Or correct NaN value using the Winsorizing approach with a reasonable approach to the correct percentile.

In [ ]:
def statistics(variable):
    stats = {}
    stats['mean'] = variable.mean()
    stats['std'] = variable.std()
    stats['max'] = variable.max()
    stats['min'] = variable.min()
    stats['median'] = np.nanmedian(variable)  #NANmedian invece di median
    stats['1%'] = np.nanpercentile(variable, 1)
    stats['5%'] = np.nanpercentile(variable, 5)
    stats['10%'] = np.nanpercentile(variable,10)  #NANpercentile invece di percentile
    stats['17,5%'] = np.nanpercentile(variable,17.5)
    stats['25%'] = np.nanpercentile(variable,25)  #NANpercentile invece di percentile
    stats['75%'] = np.nanpercentile(variable,75)  #NANpercentile invece di percentile
    stats['95%'] = np.nanpercentile(variable, 95)
    stats['99%'] = np.nanpercentile(variable,99)  #NANpercentile invece di percentile
    stats['99.5%'] = np.nanpercentile(variable,99.5)  #NANpercentile invece di percentile
    stats['99.9%'] = np.nanpercentile(variable,99.9)  #NANpercentile invece di percentile
    stats['mad'] = median_abs_deviation(variable, nan_policy = 'omit')
    return stats

#### Starting from Trip Seconds...

In [ ]:
stat_Trip_Seconds = statistics(df.Trip_Seconds)
stat_Trip_Seconds

Max value of the Trip duration (in seconds) is 85320 seconds, or 23.7h. This does not make any sense and we think that is an outlier.

For this columns we consider a Winsorizing or Trimming -
We discard values above 99.5th percentiles since they are over 73 min.

We do not consider values below 10th percentiles because trips below 3 min are meaningless.

In [ ]:
#Winsorizing of values below the 10th percentile
df.Trip_Seconds = df.Trip_Seconds.clip(lower = stat_Trip_Seconds['10%'])
#Trimming of values above the 99.5th percentile
df = df[df.Trip_Seconds < stat_Trip_Seconds['99.5%']]

df.shape

In [ ]:
#Visualizing stats after Winsorizing/Trimming
stat_Trip_Seconds_after = statistics(df.Trip_Seconds)
stat_Trip_Seconds_after

#### Now the Trip Miles

In [ ]:
stat_Trip_Miles = statistics(df.Trip_Miles)
stat_Trip_Miles

We see that Max = 980 Miles is clearly an outlier. It's grater than the distance needed to go from Chicago to New York.

Winsorizing: < 25th percentile equal to the 25th percentile because trips are less than 0.5 Miles.

Values above 99.9th percentile are instead discarded (study area is about 40km wide on its longest axis).

In [ ]:
#Winsorizing of values below the 25th percentile
df.Trip_Miles = df.Trip_Miles.clip(lower = stat_Trip_Miles['25%'])
#Trimming of values above the 99.9th percentile
df = df[df.Trip_Miles < stat_Trip_Miles['99.9%']]

df.shape

In [ ]:
#Visualizing stats after Winsorizing/Trimming
stat_Trip_Miles_after = statistics(df.Trip_Miles)
stat_Trip_Miles_after

#### Fares

In [ ]:
statistics(df.Fare)

Winsorizing at 1,1%: exludes below 1st percentile (which is the Base Fare as written on the Chicago.gov website).
Above the 99.9th percentile (higher values are probably caused by outliers from Miles and Trips).


#### Tips

In [ ]:
statistics(df.Tips)

 This is a subjetive value of each people so we think that is better
 to just ignore values above the 99.9th percentile.

#### Tolls

In [ ]:
statistics(df.Tolls)

 This is influnced by the trips outside Chicago we need to reanalyze  after

#### Extras

In [ ]:
statistics(df.Extras)

#### Trip Total

In [ ]:
statistics(df.Trip_Total)

In [ ]:
#solo colonne su cui ha senso fare media, mediana, ecc...
#num_features = [4,5,10,11,12,13,14]
#num_features = ['Trip_Seconds','Trip_Miles','Fare','Tips','Tolls','Extras','Trip_Total']

In [ ]:
#def summarize(data, features):
    #summary = {}
    #for i in range(6):
        #summary[i] = statistics(data.iloc[:,features[i]])
    #return summary

In [ ]:
#summarize(df,num_features)

# Analyzing Weather data
 We now open the dataset of weather to simply control it

In [ ]:
weather = pd.read_csv(Path("data/OHare_Airport_Daily_Weather_2020.csv"))

weather.columns = [c.replace(' ', '_') for c in weather.columns]

In [ ]:
weather.shape

In [ ]:
weather.head()

In [ ]:
weather.columns

We can se that thet columns are not so legible so we rewrite them
following the documentation of the dataset on Website

In [ ]:
weather.columns = [c.replace('AWND', 'Average_Daily_Wind_speed') for c in weather.columns]
weather.columns = [c.replace('PRCP', 'Precipitation') for c in weather.columns]
weather.columns = [c.replace('SNWD', 'Snow_depth') for c in weather.columns]
weather.columns = [c.replace('TMAX', 'Max_temperature') for c in weather.columns]
weather.columns = [c.replace('TMIN', 'Minimum temperature') for c in weather.columns]
weather.columns = [c.replace('SNOW', 'Snowfall') for c in weather.columns]
weather.columns = [c.replace('TAVG', 'Average_temperature') for c in weather.columns]
weather.columns = [c.replace('WDF5', 'Direction_of_fastest_5_second_wind') for c in weather.columns]
weather.columns = [c.replace('WSF5', 'Fastest_5_second_wind_speed') for c in weather.columns]
weather.columns = [c.replace('WT', 'Weather_Type_Value_') for c in weather.columns]

In [ ]:
weather.columns

Now we count the Nan value

In [ ]:
weather.isnull().sum()

We found that the are a lot of NaN value in the Weather
Type columns.

This follow the documentation of the datest because
the Weather Type Value can be true not for all type of weather.

In [ ]:
weather.dropna(how='all').shape

We can assume that there are not errors in the dataset of weather